In [1]:
file = spark.sparkContext.textFile("file:///C:/Users/Ashwini/Desktop/XYZBestFlight")

In [2]:
file1 = file.map(lambda row:row.split())

In [56]:
file1 = file1.filter(lambda row:len(row)==8)\
             .filter(lambda row:len(row[0])==3)\
             .filter(lambda row:len(row[1])==3)

In [58]:
file1.take(5)

[['ABE', 'ATL', 'AM', '2008-01-02', 'EV', '4923', '06:00', '-19.00'],
 ['ABE', 'ATL', 'AM', '2008-01-03', 'EV', '4877', '06:40', '409.00'],
 ['ABE', 'ATL', 'AM', '2008-01-04', 'OH', '4999', '06:40', '37.00'],
 ['ABE', 'ATL', 'AM', '2008-01-05', 'OH', '4999', '06:40', '3.00'],
 ['ABE', 'ATL', 'AM', '2008-01-06', 'OH', '4999', '06:40', '-13.00']]

In [4]:
from pyspark.sql import Row
rdd = file1.map(lambda row: Row(src=row[0], dest=row[1], am_pm=row[2], dep_date=row[3],\
                                carrier=row[4],flight_num=row[5],departure_time=row[6],arr_delay=row[7]))


In [5]:
df = spark.createDataFrame(rdd)

In [6]:
from cassandra.cluster import Cluster

In [7]:
cluster = Cluster(['127.0.0.1'])  # provide contact points and port
session = cluster.connect('aviation')

In [59]:
df.show()

+-----+---------+-------+----------+--------------+----+----------+---+
|am_pm|arr_delay|carrier|  dep_date|departure_time|dest|flight_num|src|
+-----+---------+-------+----------+--------------+----+----------+---+
|   AM|   -19.00|     EV|2008-01-02|         06:00| ATL|      4923|ABE|
|   AM|   409.00|     EV|2008-01-03|         06:40| ATL|      4877|ABE|
|   AM|    37.00|     OH|2008-01-04|         06:40| ATL|      4999|ABE|
|   AM|     3.00|     OH|2008-01-05|         06:40| ATL|      4999|ABE|
|   AM|   -13.00|     OH|2008-01-06|         06:40| ATL|      4999|ABE|
|   AM|   -14.00|     OH|2008-01-07|         06:40| ATL|      4999|ABE|
|   AM|     3.00|     OH|2008-01-08|         06:40| ATL|      4999|ABE|
|   AM|    -3.00|     OH|2008-01-09|         06:40| ATL|      4999|ABE|
|   AM|     0.00|     OH|2008-01-10|         06:40| ATL|      4999|ABE|
|   AM|     5.00|     OH|2008-01-11|         06:40| ATL|      4999|ABE|
|   AM|    -9.00|     OH|2008-01-12|         06:40| ATL|      49

In [72]:
df.write\
.format("org.apache.spark.sql.cassandra")\
.mode("append")\
.options(table="best_flight",keyspace="aviation")\
.save()

In [99]:
#date input is in the format yyyy-mm-dd
query1 = "select * from best_flight \
where src='LAX' and dest='ORD' and dep_date='2008-01-01' and am_pm='AM'"

In [100]:
result1 = session.execute(query1)

In [90]:
for i in result1:
    print("Best Flight Timings : "+i[0]+ " to "+i[1]+\
          " on "+str(i[3]) + " : "+ i[7] + "(FlightNum) " + i[5] + "(Carrier) at " \
          +str(i[6])+" "+i[2]+ " with minimum arrival delay of approximately "+str(i[4])+" minutes")

Best Flight Timings : DFW to ORD on 2008-06-10 : 1104(FlightNum) UA(Carrier) at 07:00 AM with minimum arrival delay of approximately -21.00 minutes


In [101]:
query2 = "select * from best_flight \
where src='ORD' and dest='JFK' and dep_date='2008-01-03' and am_pm='PM'"

In [102]:
result2 = session.execute(query2)

In [103]:
for i in result1:
    print("Best Flight Timings : "+i[0]+" to "+i[1]+\
          " on "+str(i[3]) + " : "+ i[7] + "(FlightNum) " + i[5] + "(Carrier) at " \
          +str(i[6])+" "+i[2]+ " with minimum arrival delay of approximately "+str(i[4])+" minutes")

for i in result2:
    print("Best Flight Timings : "+i[0]+ " to "+i[1]+\
          " on "+str(i[3]) + " : "+ i[7] + "(FlightNum) " + i[5] + "(Carrier) at " \
          +str(i[6])+" "+i[2]+ " with minimum arrival delay of approximately "+str(i[4])+" minutes")

Best Flight Timings : LAX to ORD on 2008-01-01 : 944(FlightNum) UA(Carrier) at 07:05 AM with minimum arrival delay of approximately 1.00 minutes
Best Flight Timings : ORD to JFK on 2008-01-03 : 918(FlightNum) B6(Carrier) at 19:00 PM with minimum arrival delay of approximately -7.00 minutes
